# Import default libraries

In [ ]:
# import pandas as pd
# import re

# # library to extract files with only a certain extension
# import os
# import sys

# Select URL (oai endpoints)

In [ ]:
# SOCIALHISTORYSERVICES OAI (public)
# URL = 'http://api.socialhistoryservices.org/solr/all/oai' #general, downloads ALL records
## needs set= to select between authorities or bibliographic records
## set='iish.evergreen.biblio'
## set='iish.evergreen.authority'

# EVERGREEN (on site only)
URL = 'https://evergreen.iisg.amsterdam/opac/extras/oai/authority' #authorities only
# URL = 'https://evergreen.iisg.amsterdam/opac/extras/oai/biblio' #bibliographic records only

# Test with pyoai

In [ ]:
# install https://pypi.org/project/pyoai/#files --> I already installed it, so, I comment this line
# pip install pyoai

In [ ]:
# PYOAI (https://pypi.org/project/pyoai/)
from oaipmh.client import Client
from oaipmh.metadata import MetadataRegistry, oai_dc_reader

In [ ]:
registry = MetadataRegistry()
registry.registerReader('marcxml', oai_dc_reader)
client = Client(URL, registry)

In [ ]:
# for record in client.listRecords(metadataPrefix='marcxml', set='iish.evergreen.authority'):
for record in client.listRecords(metadataPrefix='marcxml'):
    # the array indicates the part of the record: header or metadata. The field names can be used after the dot
#     print(record[0].identifier())
    print(record)

In [ ]:
# file_name = 'test_export_12may.xml' # export 12 May 2023 of all records from socialhistoryservices authorities
file_name = 'export_20230516_evergreen_pyoai.xml' # export of all records from evergreen oai authorities

# export records
with open(file_name, 'wb') as fp:
    for record in client.listRecords(metadataPrefix='marcxml'):
        fp.write(record)

conclusion of the test:
- This library has limited documentation and examples.
- I dind't manage to fetch the records, it gives a response, but it's not the record data.
- I managed to find an example about how to call the specific part of the output data --> (record[0].identifier()) calls the header and gets the identifier. However, I didn't manage to get data from array [1].metadata. 
- for socialhistory...: I am not sure why the call only retrieves 156 items (at least in display), even though the last one has an Id that seems to be the last Id? 2042661. 
- for evergreen: same issue. Via the direct call in the browser, only 50 items are retrieved, so, the call here does retrieve more items, but apparently only a sample from begin/middle/end?
- No idea how to filter...

# Test Polymatheia library

In [ ]:
# install: https://pypi.org/project/polymatheia/#files
# pip install polymatheia

In [ ]:
# POLYMATHEIA (https://polymatheia.readthedocs.io/en/latest/recipes/oai-pmh.html)
# finding available sets
from polymatheia.data.reader import OAISetReader
# finding the available MetadataFormats
from polymatheia.data.reader import OAIMetadataFormatReader
# fetching records
from polymatheia.data.reader import OAIRecordReader

# basic filters
from polymatheia.filter import RecordsFilter

In [ ]:
# Finding the available Sets
reader = OAISetReader(URL)

for setSpec in reader:
    print(setSpec)

In [ ]:
# Finding the available MetadataFormats
reader = OAIMetadataFormatReader(URL)
for format in reader:
    print(format)

In [ ]:
# Fetching records (to get them all, delete parameter max_records)
reader = OAIRecordReader(URL, max_records = 3)
for record in reader:
    print(record)

In [ ]:
# Selecting by set / (to get them all, delete parameter max_records)

## Socialhistoryservices
reader = OAIRecordReader(URL, set_spec='iish.evergreen.biblio', max_records = 3)

## Evergreen oai
# reader = OAIRecordReader(URL, set_spec='author') ## for evergreen


for record in reader:
    print(record)

In [ ]:
# Selecting the metadata format (to get them all, delete parameter max_records)

## Socialhistoryservices
reader = OAIRecordReader(URL, max_records=3, set_spec='iish.evergreen.biblio', metadata_prefix='marcxml') # for socialhistory services

## Evergreen
# reader = OAIRecordReader(URL, max_records=3, set_spec='author', metadata_prefix='marcxml') # for evergreen
# reader = OAIRecordReader(URL, max_records=3, metadata_prefix='marcxml') # for evergreen (complete)

for record in reader:
    print(record)

In [ ]:
type(record)

In [ ]:
# # applying filters (test failed...)

# reader = OAIRecordReader(URL, max_records=10, set_spec='iish.evergreen.biblio', metadata_prefix='oai_dc')
# fltr = ('contains', ['dc_subject'], 'Women')
# images = RecordsFilter(reader, fltr)
# for record in images:
#     print(record)

In [ ]:
# file_name = 'test_export_12may.xml' # export 12 May 2023 of all records from socialhistoryservices authorities
file_name = 'export_20230516_evergreen_polymatheia.xml' # export of all records from evergreen oai authorities

# export records
with open(file_name, 'wb') as fp:
    fp.write(reader)

Test conclusion:
- this library is well documented!
- it works for both services (authorities)
- Pending to test for records

# Test with Sickle

In [ ]:
# install https://sickle.readthedocs.io/en/latest/installation.html --> I already installed it, so, I comment this line
# pip install sickle

In [ ]:
# SICKLE (https://sickle.readthedocs.io/en/latest/index.html)
from sickle import Sickle
from sickle.iterator import OAIResponseIterator

In [ ]:
# Initialize an OAI interface
sickle = Sickle(URL)

In [ ]:
# Issuing requests (to get them all, delete parameter max_records)

## Socialhistoryservices
# records = sickle.ListRecords(metadataPrefix='marcxml', max_records = 1, set='iish.evergreen.authority', ignore_deleted=True) ##GOOD ONE
records = sickle.ListRecords(metadataPrefix='marcxml', max_records = 1, set='iish.evergreen.biblio', ignore_deleted=True) ##GOOD ONE

## Evergreen
# records = sickle.ListRecords(metadataPrefix='marcxml', ignore_deleted=True) #for Evergreen

print(records)

In [ ]:
# Consecutive harvesting
records.next()

In [ ]:
# getting a single record

# Socialhistoryservices
# test_record = sickle.GetRecord(identifier='oai:socialhistoryservices.org:iish.evergreen.authority:571330', metadataPrefix='marcxml')
test_record = sickle.GetRecord(identifier='oai:socialhistoryservices.org:1292895', metadataPrefix='marcxml')


# Evergreen
# test_record = sickle.GetRecord(identifier='oai:evergreen.iisg.amsterdam:527', metadataPrefix='marcxml') # Evergreen oai authority
# test_record = sickle.GetRecord(identifier='oai:evergreen.iisg.amsterdam:8', metadataPrefix='marcxml') # Evergreen oai biblio record


print(test_record)

In [ ]:
# harvesting OAI responses (save the whole XML response returned by the server: passign the iterator)
sickle = Sickle(URL, iterator=OAIResponseIterator)

In [ ]:
# Getting responses (to get them all, delete parameter max_records)

## Socialhistoryservices
# responses = sickle.ListRecords(metadataPrefix='marcxml', max_records = 1, set='iish.evergreen.authority', ignore_deleted=True) # socialhistoryservices, authorities
responses = sickle.ListRecords(metadataPrefix='marcxml', max_records = 1, set='iish.evergreen.biblio', ignore_deleted=True) # socialhistoryservices, biblio

## Evergreen
# responses = sickle.ListRecords(metadataPrefix='marcxml', ignore_deleted=True) # Evergreen

# for record in responses:
#     print(record.raw.encode('utf8'))

In [ ]:
responses.next()

In [ ]:
# file_name = 'test_export_12may.xml' # export 12 May 2023 of all records from socialhistoryservices authorities
# file_name = 'export_20230516_evergreen.xml' # export of all records from evergreen oai authorities
# file_name = 'export_20230516_evergreen_biblio.xml' # export of all records from evergreen oai bibliographic records
file_name = 'export_20230516_sohiserv_biblio_fromhome.xml' # export of all records from evergreen oai bibliographic records

# export records
with open(file_name, 'wb') as fp:
    for record in responses:
        fp.write(record.raw.encode('utf8'))

## other libraries to test

In [ ]:
#install https://pypi.org/project/oaiharvest/ --> I already installed it, so, I comment this line
#pip install http://github.com/bloomonkey/oai-harvest.git #egg=oaiharvest

# install https://github.com/bloomonkey/oai-harvest
# pip install oaiharvest

In [ ]:
# TO TEST: Recommended by Rik Hoekstra
# pyoaiharvester (https://github.com/vphill/pyoaiharvester)

In [ ]:
# TO TEST: Recommended by Menzo Windhouwer
# Hi 
# @Liliana Melgar
# , we (
# @Vic
#  and I) have extensive experience with OAI-PMH harvesting using https://github.com/clarin-eric/oai-harvest-manager which is Java based not Python.